# Hausholder transformation project 
##### made  for numerical analysis classes by Kamil Bartocha, Marcin Baranek and Mateusz Kamyczura

### Import Libraries

In [25]:
import numpy as np
import math
import time

### Input matrix from user

In [26]:
def input_matrix():
    # Take matrix size as input
    while True:
        try:
            row_number = int(input("enter number of rows: "))
            column_number = int(input("enter number of columns: "))
            break
        except ValueError:
            print("That was no valid number. Try to input integer again ")
        
    # initialise matrix with zeroes
    matrix = np.zeros((row_number, column_number))

    # input each element row at a time
    for row_iterator in range(row_number):
        for column_iterator in range(column_number):
            while True:
                try:
                    matrix[row_iterator][column_iterator] = input(f'enter value of matrix[{row_iterator}][{column_iterator}] = ')
                    break
                except ValueError:
                    print("That was no valid number. Try to input number again ")   
    return matrix

In [27]:
matrix = input_matrix()
#print(matrix)

enter number of rows: 1
enter number of columns: 1
enter value of matrix[0][0] = 2


### Householder transformation 

In [28]:
def householder_transformation_matrix(vector: np.ndarray, new_vector: np.ndarray,
                                      time_of_execution=False) -> np.ndarray:
    if time_of_execution:
        now = time.time()
    if not (isinstance(vector, np.ndarray) and isinstance(new_vector, np.ndarray)):
        print("Error in householder_transformation: input vector is not a numpy.ndarray object")
        return np.array(False)
    vector = np.reshape(vector, newshape=(-1, 1))
    new_vector = np.reshape(new_vector, newshape=(-1, 1))
    if vector.shape != new_vector.shape:
        print("Error in householder_transformation: wrong shape of inputs vectors")
        return np.array(False)
    dimensional = vector.shape[0]
    cos_of_angel_between_vector_and_new_vector = np.dot(np.transpose(vector), new_vector)
    #for numerical stability we chose a vector of more norm
    if cos_of_angel_between_vector_and_new_vector >= 0:
        householder_vector = vector + math.sqrt((vector**2).sum()) * new_vector
    else:
        householder_vector = vector - math.sqrt((vector ** 2).sum()) * new_vector
    specular_reflection = np.dot(np.transpose(householder_vector), householder_vector)
    if specular_reflection != 0:
        specular_reflection = 2 / specular_reflection
    else:
        print("Error in householder_transformation: specular reflection is equal +infinity")
        print("you tried to divide by 0")
        return np.array(False)
    identity_matrix = init_identity_matrix(dimensional)
    #create householder matrix
    householder_matrix = specular_reflection * np.dot(householder_vector, np.transpose(householder_vector))
    householder_matrix = identity_matrix - householder_matrix
    if time_of_execution:
        print(f"time of execution householder_transformation_matrix is: {time.time() - now} s")
    return householder_matrix

### improvement r matrix
###### for numerical stability, inserts 0 where they sure are (for upper triangular matrix)

In [29]:
def improvement_r_matrix(matrix: np.ndarray, iteration: int) -> np.ndarray:
    j = 1
    for i in range(iteration):
        matrix[j:, i] = np.zeros(shape=matrix[j:, i].shape)
        j += 1
    return matrix

### improvement matrix
###### for numerical stability, inserts $\epsilon$ where they sure are (for upper triangular matrix)

In [30]:
def improvement_matrix(matrix: np.ndarray, epsilon=1.e-5):
    matrix = matrix.astype(dtype=float)
    location = np.where(abs(matrix - 0) < epsilon)
    matrix[location] = epsilon
    return matrix

### Hausholder algorithm

In [34]:
def householder_algorithm(matrix: np.ndarray, improve_matrix=True,
                          improve_r_matrix=True, time_of_execution=False) -> (np.ndarray, np.ndarray):
    if time_of_execution:
        now = time.time()
    if not isinstance(matrix, np.ndarray):
        print("Error in householder_algorithm: input matrix is not a numpy.ndarray object")
        return False
    if improve_matrix:
        matrix = improvement_matrix(matrix)
    first_dimensional = matrix.shape[0]
    if first_dimensional > matrix.shape[1]:
        print("The matrix has the first dimension greater than the second,\n"
              "therefore the algorithm was run for the transposed matrix")
        first_dimensional = matrix.shape[1]
        matrix = np.transpose(matrix)
    identity_matrix = init_identity_matrix(first_dimensional)
    # create P, Q and R matrix
    # Q matrix will be transpose be for end of function
    p_matrix = identity_matrix
    q_matrix = np.zeros(shape=p_matrix.shape)
    r_matrix = matrix
    # create unit vector
    unit_vector = np.zeros(shape=(first_dimensional, 1)).astype(int)
    unit_vector[0, 0] = 1
    for i in range(first_dimensional):
        householder_matrix = householder_transformation_matrix(r_matrix[i:, i], unit_vector)
        if not householder_matrix.all():
            print("something went wrong")
            print("check the assumptions")
            return False, False
        # preparing unit vector to next iteration
        unit_vector = np.reshape(np.delete(unit_vector, -1), newshape=(-1, 1))
        # update P matrix
        p_matrix[i:, i:] = householder_matrix
        # update R matrix
        r_matrix = np.dot(p_matrix, r_matrix)
        if improve_r_matrix:
            r_matrix = improvement_r_matrix(r_matrix, i)
        # update Q matrix
        if i == 0:
            q_matrix = p_matrix
        else:
            q_matrix = np.dot(p_matrix, q_matrix)
        # preparing P matrix to next iteration
        p_matrix = init_identity_matrix(first_dimensional)
    q_matrix = np.transpose(q_matrix)
    if time_of_execution:
        print(f"time of execution householder_algorithm is: {time.time() - now} s")
    return q_matrix, r_matrix

### Linear equations with using hausholder algorithm

In [35]:
def linear_equations_with_householder_algorithm(matrix: np.ndarray, bias: np.ndarray) -> np.ndarray:
    dim = matrix.shape[0]
    bias = np.reshape(bias, newshape=(dim, 1))
    q_matrix, r_matrix = householder_algorithm(matrix)
    bias = np.dot(np.transpose(q_matrix), bias)
    solve = np.array(bias[-1, 0] / r_matrix[-1, -1])
    solve = np.reshape(solve, newshape=(-1, 1))
    for i in range(1, dim, 1):
        next_solve_value = (bias[dim - i - 1, 0] - np.dot(r_matrix[dim - i - 1, dim - i:],
                                                          solve))\
                           / r_matrix[dim - i - 1, dim - i - 1]
        solve = np.append(next_solve_value, solve)
        solve = np.reshape(solve, newshape=(-1, 1))
    return solve